# Tutorial 1: Basic C(t) Calculation

## Introduction
This tutorial demonstrates how to:
1. Create a neural network
2. Simulate brain-like dynamics
3. Calculate consciousness measure C(t)
4. Visualize results

## Theory
The consciousness measure is defined as:
$$C(t) = \Phi(t) \times R(t) \times [1 - e^{-\lambda D(t)}]$$

Where:
- $\Phi(t)$: Integrated information (bits)
- $R(t)$: Recursive depth (dimensionless)
- $D(t)$: Inverse distance from criticality (0-1)
- $\lambda$: Sensitivity parameter (default: 5.0)

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from ouroboros_observer import NeuralNetwork, ConsciousnessMeasure
from ouroboros_observer.visualization import plots

# Set random seed for reproducibility
np.random.seed(42)

## Step 1: Create Neural Network

In [ ]:
# Create a network with 100 neurons
N_neurons = 100
network = NeuralNetwork(
    N_neurons=N_neurons,
    dt=0.1,  # Time step: 0.1 ms
    seed=42
)

print(f"Created network with {N_neurons} neurons")
print(f"Connectivity: {network.W.sum():.0f} synapses")
print(f"Connection density: {network.W.sum() / (N_neurons * (N_neurons-1)):.1%}")

## Step 2: Run Simulation

In [ ]:
# Run simulation with moderate external input
duration = 5000.0  # 5 seconds
I_ext = np.random.randn(N_neurons) * 2.0 + 5.0

results = network.run(duration=duration, I_ext=I_ext)

print(f"Simulation complete: {duration} ms")
print(f"Total spikes: {results['spikes'].sum():.0f}")
print(f"Mean firing rate: {results['spikes'].sum() / (N_neurons * duration / 1000):.2f} Hz")

## Step 3: Calculate C(t)

In [ ]:
# Initialize consciousness measure calculator
measure = ConsciousnessMeasure(
    lambda_param=5.0,
    Phi_method='geometric',  # Fast method
    seed=42
)

# Calculate C(t) over time (every 50 ms)
sample_interval = 50
C_timeseries = []
Phi_timeseries = []
R_timeseries = []
D_timeseries = []
time_points = []

for i in range(sample_interval, len(results['spikes']), sample_interval):
    activity = results['spikes'][i]
    history = results['spikes'][max(0, i-100):i]
    
    C, components = measure.calculate_C(
        activity=activity,
        connectivity=network.W,
        activity_history=history
    )
    
    C_timeseries.append(C)
    Phi_timeseries.append(components['Phi'])
    R_timeseries.append(components['R'])
    D_timeseries.append(components['D'])
    time_points.append(results['time'][i])

C_timeseries = np.array(C_timeseries)
Phi_timeseries = np.array(Phi_timeseries)
R_timeseries = np.array(R_timeseries)
D_timeseries = np.array(D_timeseries)
time_points = np.array(time_points)

print(f"\nConsciousness Measure Statistics:")
print(f"  Mean C(t): {C_timeseries.mean():.2f} Â± {C_timeseries.std():.2f} bits")
print(f"  Max C(t): {C_timeseries.max():.2f} bits")
print(f"  Min C(t): {C_timeseries.min():.2f} bits")
print(f"  State: {'CONSCIOUS' if C_timeseries.mean() > 8.3 else 'UNCONSCIOUS'}")

## Step 4: Visualize Results

In [ ]:
# Plot C(t) time series
fig, ax = plots.plot_C_timeseries(
    C_timeseries,
    time=time_points,
    critical_threshold=8.3,
    title="Consciousness Measure Over Time"
)
plt.show()

In [ ]:
# Plot all components
fig, axes = plots.plot_components(
    Phi_timeseries,
    R_timeseries,
    D_timeseries,
    C_timeseries,
    time=time_points
)
plt.show()

In [ ]:
# Plot network activity
fig, axes = plots.plot_network_activity(
    results['spikes'][:2000],  # First 2 seconds
    connectivity=network.W,
    figsize=(14, 6)
)
plt.show()

## Step 5: Phase Space Visualization

In [ ]:
# 3D phase space trajectory
fig, ax = plots.plot_phase_space(
    Phi_timeseries,
    R_timeseries,
    D_timeseries,
    C_t=C_timeseries
)
plt.show()

## Interpretation

**Key Observations:**
1. **C(t) value**: If C(t) > 8.3 bits â†’ system in conscious state
2. **Î¦ component**: Measures integration across network
3. **R component**: Indicates depth of self-referential processing
4. **D component**: Shows proximity to critical dynamics

**What affects C(t)?**
- External input strength
- Network connectivity structure
- Synaptic strengths
- Distance from criticality

## Exercise: Experiment with Parameters

Try modifying:
1. Network size (N_neurons)
2. External input strength (I_ext)
3. Simulation duration
4. Lambda parameter in ConsciousnessMeasure

Observe how these affect C(t) and its components!